In [1]:
# Step 1: Environment Setup and Module Import
# ------------------------------------------------------------------------------
# Import the necessary libraries and modules to run the script smoothly.
# Add the project's root path to the system path so Python can find the QuantBT library.
# ------------------------------------------------------------------------------
import sys
import os
import asyncio
from pathlib import Path
from typing import List, Dict, Any
from datetime import datetime

# Set project root path so Python can find the QuantBT library.
# This code reliably finds the root path regardless of script execution location.
try:
    current_dir = Path(os.path.dirname(os.path.realpath(__file__)))
except NameError:
    current_dir = Path.cwd()

project_root = current_dir.parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import the core components of QuantBT.
import polars as pl
from quantbt.core.interfaces.strategy import TradingStrategy
from quantbt.core.value_objects.backtest_config import BacktestConfig
from quantbt import OrderSide, OrderType
from quantbt import Order
from quantbt.ray.bayesian_parameter_optimizer import BayesianParameterOptimizer
from quantbt.ray.optimization.parameter_space import ParameterSpace

In [2]:
# Step 2: Define trading strategy
# ------------------------------------------------------------------------------
# Define the trading strategy to optimize.
# In this example, we use 'SimpleSMAStrategy' which uses two Simple Moving Averages (SMA).
#
# You must inherit from the TradingStrategy class and implement the following two core methods:
# 1. _compute_indicators_for_symbol: Pre-compute indicators using Polars in a vectorized manner.
#    This is executed only once before backtesting starts and is very fast.
# 2. generate_signals: Generate trading signals for each time step (row) based on the computed indicators.
# ------------------------------------------------------------------------------
class SimpleSMAStrategy(TradingStrategy):
    """
    Simple Moving Average (SMA) crossover strategy (for Ray optimization)
    - Buy signal: When short-term SMA (buy_sma) crosses above long-term SMA (sell_sma) (simplified as price > buy_sma)
    - Sell signal: When price falls below long-term SMA (sell_sma)
    """
    def __init__(self, buy_sma: int = 15, sell_sma: int = 30, position_size_pct: float = 0.8):
        # Initialize the basic settings of the strategy.
        super().__init__(
            name="SimpleSMAStrategy",
            position_size_pct=position_size_pct,  # Invest 80% of assets at once
            max_positions=1         # Hold a maximum of 1 position
        )
        self.buy_sma = buy_sma
        self.sell_sma = sell_sma
        self.position_size_pct = position_size_pct

    def _compute_indicators_for_symbol(self, symbol_data: pl.DataFrame) -> pl.DataFrame:
        """
        Calculate moving average indicators for each symbol. (Polars vector operations)
        This method is called only once by the backtesting engine after data loading.
        """
        # Sort data chronologically.
        data = symbol_data.sort("timestamp")

        # Use Polars' built-in functions to calculate SMA quickly and efficiently.
        buy_sma_series = self.calculate_sma(data["close"], self.buy_sma)
        sell_sma_series = self.calculate_sma(data["close"], self.sell_sma)

        # Add the calculated indicators as new columns to the dataframe.
        buy_sma_name = f"sma_{self.buy_sma}"
        sell_sma_name = f"sma_{self.sell_sma}"

        # Handle to prevent duplicate column names.
        columns_to_add = [buy_sma_series.alias(buy_sma_name)]
        if sell_sma_name != buy_sma_name:
            columns_to_add.append(sell_sma_series.alias(sell_sma_name))

        return data.with_columns(columns_to_add)

    def generate_signals(self, current_data: Dict[str, Any]) -> List[Order]:
        """
        Generate trading signals based on data from each time step (row).
        This method is called row by row in the backtest loop.
        """
        orders = []
        if not self.broker:
            return orders

        # Extract the necessary values from the current data point.
        symbol = current_data['symbol']
        current_price = current_data['close']
        buy_sma_val = current_data.get(f'sma_{self.buy_sma}')
        sell_sma_val = current_data.get(f'sma_{self.sell_sma}', buy_sma_val)

        # Skip the initial period where indicator values are not yet calculated.
        if buy_sma_val is None or sell_sma_val is None:
            return orders

        # Check the current position status.
        has_position = self.get_current_positions().get(symbol, 0) > 0

        # Buy signal: When current price exceeds 'buy_sma' and there's no position held
        if current_price > buy_sma_val and not has_position:
            portfolio_value = self.get_portfolio_value()
            quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
            if quantity > 0:
                orders.append(Order(symbol, OrderSide.BUY, quantity, OrderType.MARKET))

        # Sell signal: When current price falls below 'sell_sma' and there's a position held
        elif current_price < sell_sma_val and has_position:
            quantity = self.get_current_positions()[symbol]
            orders.append(Order(symbol, OrderSide.SELL, quantity, OrderType.MARKET))

        return orders

In [3]:
# Step 3: Backtest Environment Configuration (BacktestConfig)
# --------------------------------------------------------------------------
# Set up the basic environment required for backtesting.
# Define which assets to test, for what period, with how much capital, etc.
# --------------------------------------------------------------------------
print("Step 3: Setting up backtest environment...")
config = BacktestConfig(
    symbols=["KRW-BTC"],                  # Target asset for optimization
    start_date=datetime(2024, 1, 1),    # Start date
    end_date=datetime(2024, 3, 31),  # End date
    timeframe="1d",                         # Time frame (daily candles)
    initial_cash=10_000_000,              # Initial capital (10 million KRW)
    commission_rate=0.0,               # Commission fee
    slippage_rate=0.0,                 # Slippage
    save_portfolio_history=False,    
)
print("✅ Backtest environment setup completed.")
print("-" * 70)

Step 3: Setting up backtest environment...
✅ Backtest environment setup completed.
----------------------------------------------------------------------


In [4]:
# Step 4: Define Parameter Search Space (ParameterSpace)
# --------------------------------------------------------------------------
# Define the parameters to optimize and their search ranges.
# The Bayesian optimizer will find the optimal parameter combination within this space.
# Using the from_dict class method allows you to conveniently define the space.
#   - (min_value, max_value) tuple: Integer or Real parameters
#   - [...] list: Categorical parameters
# --------------------------------------------------------------------------
print("Step 4: Defining parameter search space...")
param_config = {
    'buy_sma': (1, 50),   # 'buy_sma' is an integer between 1 and 50
    'sell_sma': (1, 50),   # 'sell_sma' is an integer between 1 and 50
    'position_size_pct': (0.5, 1),   # 'position_size_pct' is a real number between 0.5 and 1
}
param_space = ParameterSpace.from_dict(param_config)
print("✅ Parameter search space definition completed:")
for name, dim in zip(param_space.dimension_names, param_space.dimensions):
    print(f"   - Parameter '{name}': {dim}")
print("-" * 70)

Step 4: Defining parameter search space...
✅ Parameter search space definition completed:
   - Parameter 'buy_sma': Integer(low=1, high=50, prior='uniform', transform='identity')
   - Parameter 'sell_sma': Integer(low=1, high=50, prior='uniform', transform='identity')
   - Parameter 'position_size_pct': Real(low=0.5, high=1, prior='uniform', transform='identity')
----------------------------------------------------------------------


In [5]:
# Step 5: Create and Execute Bayesian Optimizer
# --------------------------------------------------------------------------
# Combine all components to create the BayesianParameterOptimizer.
# The optimizer internally sets up a Ray cluster, runs multiple backtests in parallel,
# and uses scikit-optimize to intelligently determine the next parameters to explore.
# --------------------------------------------------------------------------
print("Step 5: Creating Bayesian optimizer and preparing for execution...")
optimizer = BayesianParameterOptimizer(
    strategy_class=SimpleSMAStrategy,    # Strategy class to optimize
    param_space=param_space,             # Defined parameter space
    config=config,                       # Backtest environment configuration
    num_actors=16,                        # Number of CPU cores for parallel execution (Ray actors)
    n_initial_points=16                  # Number of initial random searches (Bayesian optimization starts afterwards)
)
print("✅ Optimizer creation completed.")
print("\n⏳ Starting optimization... (Progress will be displayed in real-time)")

# Execute optimization!
# objective_metric: Target metric for optimization (e.g., 'sharpe_ratio')
# n_iter: Total number of iterations
# early_stopping_patience: Early termination if no performance improvement for N iterations
results = await optimizer.optimize(
    objective_metric='sharpe_ratio',
    n_iter=100,
    early_stopping_patience=5,
    early_stopping_min_delta=0.02
)


Step 5: Creating Bayesian optimizer and preparing for execution...
✅ Optimizer creation completed.

⏳ Starting optimization... (Progress will be displayed in real-time)
=============== Bayesian Optimization Progress ===============
진행률: 0/100 (0.0%)
░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 Current Performance:
   Total Results: 0
   Success Rate: 0.0%
   Average Sharpe Ratio: 0.0000
   Average Return: 0.0000
   Average Execution Time: 0.00s


2025-06-28 00:50:15,910	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


=============== Bayesian Optimization Progress ===============
진행률: 16/100 (16.0%), ETA: 13초
████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 Current Performance:
   Total Results: 16
   Success Rate: 100.0%
   Average Sharpe Ratio: 7.5084
   Average Return: 0.3663
   Average Execution Time: 0.01s
   Best Sharpe Ratio: 14.0309 (Parameters: {'buy_sma': np.int64(36), 'sell_sma': np.int64(16), 'position_size_pct': 0.7906599883838161})
=============== Bayesian Optimization Progress ===============
진행률: 32/100 (32.0%), ETA: 16초
████████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
--------------------------------------------------------------
📊 Current Performance:
   Total Results: 32
   Success Rate: 100.0%
   Average Sharpe Ratio: 8.2225
   Average Return: 0.3814
   Average Execution Time: 0.01s
   Best Sharpe Ratio: 14.6014 (Parameters: {'buy_sma': np.int64(34), 'sell_sma': np.int64(21), 'position_size_pct': 0.8200481024737

In [6]:
# Final Results Summary
# The optimize method returns a list of results from all attempts.
# Find and display the best performing result among them.
if results:
    best_result = max(results, key=lambda x: x['result'].get('sharpe_ratio', -999))
    print("🏆 Optimal Performance Parameters and Results:")
    print(f"   - Parameters: {best_result['params']}")
    print(f"   - Sharpe Ratio: {best_result['result']['sharpe_ratio']:.4f}")
    print(f"   - Total Return: {best_result['result']['total_return']:.4f}")
    print(f"   - Maximum Drawdown (MDD): {best_result['result']['max_drawdown']:.4f}")
else:
    print("No results available.")

🏆 Optimal Performance Parameters and Results:
   - Parameters: {'buy_sma': np.int64(34), 'sell_sma': np.int64(21), 'position_size_pct': 0.8200481024737414}
   - Sharpe Ratio: 14.6014
   - Total Return: 0.5948
   - Maximum Drawdown (MDD): 0.0983
